In [3]:
# import libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from tkinter import Tk, filedialog
import os
import time
import re
import requests
import subprocess
from datetime import date
import openpyxl

from selenium import webdriver                                     # automate web browser interaction
from selenium.webdriver.common.keys import Keys                    # automate keyboard actions
from selenium.webdriver.common.by import By                        # find elements by html id on webpage
from selenium.webdriver.support.ui import Select                   # automate dropdown selection
from selenium.webdriver.support.ui import WebDriverWait            # command driver to wait
from selenium.webdriver.support import expected_conditions as EC   # command driver to wait until loaded
from selenium.webdriver.chrome.options import Options              # option to make webdriver not visible
from selenium.common.exceptions import WebDriverException          # Deal with exptions in webdriver

In [53]:
ref = pd.read_excel("glansis_references_2024-05-05.xlsx", dtype = str)
print(ref.shape)
ref.head()

(25511, 8)


,species.id,scientific.name,common.name,status,group,refnum,type,reference
0,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,40167,Report,2015. Final Report: South Carolina State Wildl...
1,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,41080,Journal Article,"Abdelmonem, A.A., M.M. Metwally, H.S. Hussein,..."
2,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,23905,Journal Article,"Aieta, A.E., and K. Oliveira. 2009. Distributi..."
3,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,41061,Journal Article,"Ashworth, S.T., and C.R. Kennedy. 1999. Densit..."
4,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,41062,Journal Article,"Ashworth, S.T., C.R. Kennedy, and G. Blanc. 19..."


In [54]:
# select only journal article references
ref = ref[ref['type'] == 'Journal Article']
print(ref['type'].unique())
print(ref.shape)
ref.head()

['Journal Article']
(13167, 8)


,species.id,scientific.name,common.name,status,group,refnum,type,reference
1,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,41080,Journal Article,"Abdelmonem, A.A., M.M. Metwally, H.S. Hussein,..."
2,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,23905,Journal Article,"Aieta, A.E., and K. Oliveira. 2009. Distributi..."
3,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,41061,Journal Article,"Ashworth, S.T., and C.R. Kennedy. 1999. Densit..."
4,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,41062,Journal Article,"Ashworth, S.T., C.R. Kennedy, and G. Blanc. 19..."
5,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,40957,Journal Article,"Audenaert, V., T. Huyse, G. Goemans, C. Belpai..."


In [55]:
ref.drop_duplicates(subset=['refnum'], keep='first', inplace=True)
print(ref.shape)
ref.head()

(10185, 8)


,species.id,scientific.name,common.name,status,group,refnum,type,reference
1,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,41080,Journal Article,"Abdelmonem, A.A., M.M. Metwally, H.S. Hussein,..."
2,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,23905,Journal Article,"Aieta, A.E., and K. Oliveira. 2009. Distributi..."
3,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,41061,Journal Article,"Ashworth, S.T., and C.R. Kennedy. 1999. Densit..."
4,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,41062,Journal Article,"Ashworth, S.T., C.R. Kennedy, and G. Blanc. 19..."
5,3830,Anguillicola crassus,Eel swim bladder parasite,nonindingenous,Nematode,40957,Journal Article,"Audenaert, V., T. Huyse, G. Goemans, C. Belpai..."


In [13]:
# start web driver
driver = webdriver.Chrome()
driver.get('https://nas.er.usgs.gov/Signin.aspx')

In [56]:
# extract headers first
time_start = time.time()

# create empty list
data = []

# function to extract table from HTML
def find_table():
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    tables = soup.find_all('table')
    ref_table = tables[33]
    return ref_table

# function to extract column data
def extract_table_data(table):
    column_data = []
    for row in table.find_all('tr'):
        
        cells = row.find_all('td')
        cell_text = cells[1].get_text().strip()
        column_data.append(cell_text)
    data.append(column_data)
        
    return data

# scrape references
for refnum in ref['refnum'].values:
    url = "https://nas.er.usgs.gov/DataEntry/References/ReferenceViewer.aspx?refnum="  + str(refnum)
    driver.get(url)
    table_data = find_table()
    extract_table_data(table_data)
    
# get headers
headers = []
for row in table_data.find_all('tr'):
    cells = row.find_all('td')
    headers.append(cells[0].get_text().strip())

# create data table    
full_ref = pd.DataFrame(data, columns = headers)

print('Full Reference Pull:')
print(round((time.time()- time_start)/60))

Impact Table Pull:
29


In [61]:
# Sanitize the data
full_ref = full_ref.applymap(lambda x: re.sub(r'[\x00-\x1F\x7F-\x9F]', '', str(x)))

C:\Users\redinger\AppData\Local\Temp\1\ipykernel_4888\2661972647.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  full_ref = full_ref.applymap(lambda x: re.sub(r'[\x00-\x1F\x7F-\x9F]', '', str(x)))


In [62]:
full_ref.to_excel('output.xlsx', index = False)